# Tools  🔨🪚🔩⚙️⚒️🧲🧰🔧🪛🔩⚙️🔦🧭🔗⛓️🧮
Tools allow agents to 'Act' in the real world. 
Careful descriptions can help your agent discover how to use your tools.

LangChain supports many tool formats and tool sets. Here we will cover some common cases, but check the [docs](https://docs.langchain.com/oss/python/langchain/tools) for more information.

## Setup

Load and/or check for needed environmental variables

In [ ]:
from dotenv import load_dotenv
from env_utils import doublecheck_env

# Load environment variables from .env
load_dotenv()

# Check and print results
doublecheck_env(".env")

## Calculator example

In this example, the docstring and inferred arguments and argument types are used by the LLM to detetermine when and how to call the tool.

In [ ]:
from typing import Literal

from langchain.tools import tool


@tool
def real_number_calculator(
    a: float, b: float, operation: Literal["add", "subtract"]
) -> float:
    """Perform addition and subtraction operations on two real numbers."""
    print("🧮 Invoking calculator tool")
    # Perform the specified operation
    if operation == "add":
        return a + b
    elif operation == "subtract":
        return a - b
    else:
        raise ValueError(f"Invalid operation: {operation}")

In [ ]:
from langchain.agents import create_agent

agent = create_agent(
    model="groq:llama-3.1-8b-instant",
    tools=[real_number_calculator],
    system_prompt="You are a helpful assistant to a banker. You help with numerical calculations. Base rate for home loan is 5.0 percent. Base rate for CD is 4.0 percent. Base rate for auto loan is 7.0 percent. ",
)

This invokes your calculator tool.

In [ ]:
result = agent.invoke(
    {"messages": [{"role": "user", "content": "Need to charge additional 75 basis points on home loan for this customer. What is the final rate?"}]}
)
print(result["messages"][-1].content)

The tool description can have a big impact. 
This may not  invoke your calculator tool because the inputs are integers. It may attempt to invoke some other unknown tool (results vary from run to run)

In [ ]:
result = agent.invoke({"messages": [{"role": "user", "content": "Base rate is 5. Apply additional margin of 1 for this customer. What is the final rate?"}]})
print(result["messages"][-1].content)

## Adding a more detailed description
While a basic description is often sufficient, LangChain has support for enhanced descriptions. The example below uses one method: Google Style argument descriptions. Used with `parse_docstring=True`, this will parse and pass the arg descriptions to the model. You can rename the tool and change its description. This can be effective when you are sharing a standard tool but would like agent-specific instructions.

In [65]:
from typing import Literal

from langchain.tools import tool


@tool(
    "calculator",
    parse_docstring=True,
    description=(
        "Perform basic arithmetic operations on two real numbers."
        "Use this whenever you have operations on any numbers, even if they are integers."
    ),
)
def real_number_calculator(
    a: float, b: float, operation: Literal["add", "subtract"]
) -> float:
    """Perform basic arithmetic operations on two real numbers.

    Args:
        a (float): The first number.
        b (float): The second number.
        operation (Literal["add", "subtract", "multiply", "divide"]):
            The arithmetic operation to perform.

            - `"add"`: Returns the sum of `a` and `b`.
            - `"subtract"`: Returns the result of `a - b`.

    Returns:
        float: The numerical result of the specified operation.

    Raises:
        ValueError: If an invalid operation is provided or division by zero is attempted.
    """
    print("🧮  Invoking calculator tool")
    # Perform the specified operation
    if operation == "add":
        return a + b
    elif operation == "subtract":
        return a - b
    else:
        raise ValueError(f"Invalid operation: {operation}")

In [66]:
from langchain.agents import create_agent

agent = create_agent(
    model="groq:llama-3.1-8b-instant",
    tools=[real_number_calculator],
    system_prompt="You are a helpful assistant to a banker. You help with numerical calculations. Base rate for home loan is 5.0 percent. Base rate for CD is 4.0 percent. Base rate for auto loan is 7.0 percent. ",
)

In [76]:
result = agent.invoke({"messages": [{"role": "user", "content": "Base rate is 5. Apply additional margin of 1 for this customer. What is the final rate?"}]})
print(result["messages"][-1].content)

🧮  Invoking calculator tool
The final rate for the home loan is 6.0 percent.


## Try your own.
Create a tool of your own and try it here!

In [ ]:

@tool
def your_tool(
    a: float, b: float, 
) -> float:
    """Perform your favorite operation

    Args:
        a (float): operator a description
        b (float): operator b description

    Returns:
        float: description
    """
    pass

In [ ]:
from langchain.agents import create_agent

agent = create_agent(
    model="groq:llama-3.1-8b-instant",
    tools=[your_tool],
    system_prompt="your system prompt",
)